In [1]:
%%bash
python3 -m pip install kafka-python rx

In [2]:
from kafka import KafkaProducer
from kafka.errors import KafkaError

producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

# Asynchronous by default
future = producer.send('my-topic', b'raw_bytes')

# Block for 'synchronous' sends
try:
    record_metadata = future.get(timeout=10)
except KafkaError:
    # Decide what to do if produce request failed...
    log.exception()
    pass

# Successful result returns assigned partition and offset
print (record_metadata.topic)
print (record_metadata.partition)
print (record_metadata.offset)

# produce keyed messages to enable hashed partitioning
producer.send('test', key=b'foo', value=b'bar')

# configure multiple retries
producer = KafkaProducer(retries=5)


my-topic
0
7


In [3]:
import pandas
df = pandas.read_json("social-pulse-milano.json")


In [5]:
df.head()

,count,items
0,269290,"{'language': 'it', 'created': '2013-11-02T00:0..."
1,269290,"{'language': 'it', 'created': '2013-11-02T00:0..."
2,269290,"{'language': 'it', 'created': '2013-11-02T00:0..."
3,269290,"{'language': 'it', 'created': '2013-11-02T00:0..."
4,269290,"{'language': 'so', 'created': '2013-11-02T00:0..."


In [6]:
from pandas.io.json import json_normalize

In [7]:
itemsdf = json_normalize(df['items'].head())
itemsdf.head()

,created,entities,geometry.coordinates,geometry.type,language,municipality.acheneID,municipality.name,timestamp,user
0,2013-11-02T00:03:08.000+01:00,[],"[9.1434764, 45.40239345]",Point,it,http://dandelion.eu/resource/f845ac8310786c717...,Assago,1383346988,b851cae05f
1,2013-11-02T00:03:20.000+01:00,[],"[9.15929745, 45.45605198]",Point,it,http://dandelion.eu/resource/2bf72cfc3857f470b...,Milano,1383347000,f1894a42af
2,2013-11-02T00:03:41.000+01:00,[],"[9.21164285, 45.48993448]",Point,it,http://dandelion.eu/resource/2bf72cfc3857f470b...,Milano,1383347021,54742f3540
3,2013-11-02T00:03:43.000+01:00,"[http://it.dbpedia.org/resource/Vodafone, http...","[9.08727888, 45.43227401]",Point,it,http://dandelion.eu/resource/57f2233142f0209fe...,Cesano Boscone,1383347023,fcd64a8f01
4,2013-11-02T00:04:00.000+01:00,[],"[9.21170695, 45.48993288]",Point,so,http://dandelion.eu/resource/2bf72cfc3857f470b...,Milano,1383347040,54742f3540


In [8]:
itemsdf["municipality.acheneID"][0]

'http://dandelion.eu/resource/f845ac8310786c717c991654e453047f1bd0d57d'

In [9]:
weatherdf = pandas.read_json("meteotrentino-weather-station-data.json")
weatherdf.head()

,count,items
0,2195,"{'maxTemperature': 12.3, 'elevation': 905, 'ge..."
1,2195,"{'maxTemperature': 10.2, 'elevation': 1155, 'g..."
2,2195,"{'maxTemperature': 11.8, 'elevation': 1205, 'g..."
3,2195,"{'maxTemperature': 13.6, 'elevation': 720, 'ge..."
4,2195,"{'maxTemperature': 16.3, 'elevation': 201, 'ge..."


In [10]:
weatherdf = json_normalize(weatherdf['items'])
weatherdf.head()

,date,elevation,geometry.coordinates,geometry.type,maxTemperature,maxWind,minTemperature,minWind,precipitation,precipitations.0000,...,winds.2130,winds.2145,winds.2200,winds.2215,winds.2230,winds.2245,winds.2300,winds.2315,winds.2330,winds.2345
0,2013-11-01,905,"[10.79582897, 46.31340453]",Point,12.3,2.5,4.5,0.0,False,0.0,...,0@183,,0@157,,0.4@228,0.3@248,,0@201,0@199,
1,2013-11-01,1155,"[11.25371981, 45.94027203]",Point,10.2,NaN,6.5,NaN,False,0.0,...,,,,,,,,,,
2,2013-11-01,1205,"[11.6645808, 46.38363633]",Point,11.8,NaN,6.5,NaN,False,0.0,...,,,,,,,,,,
3,2013-11-01,720,"[10.91841055, 46.351598010000004]",Point,13.6,4.8,6.2,0.0,False,0.0,...,0.3@198,0.3@196,0.4@236,,0.9@15,0.6@345,1@244,0.5@192,0@157,
4,2013-11-01,201,"[11.07973339, 46.15635256]",Point,16.3,NaN,11.1,NaN,False,0.0,...,,,,,,,,,,


In [11]:
weatherdf.groupby(weatherdf["geometry.coordinates"].map(tuple)).apply(list)

geometry.coordinates
(10.59682317, 46.26251044)           [date, elevation, geometry.coordinates, geomet...
(10.64240944, 45.940616059999996)    [date, elevation, geometry.coordinates, geomet...
(10.72502501, 45.89562846)           [date, elevation, geometry.coordinates, geomet...
(10.7308802, 46.04113243)            [date, elevation, geometry.coordinates, geomet...
(10.75167519, 46.0591135)            [date, elevation, geometry.coordinates, geomet...
(10.7574828, 46.15647979)            [date, elevation, geometry.coordinates, geomet...
(10.79582897, 46.31340453)           [date, elevation, geometry.coordinates, geomet...
(10.88713268, 45.91037378)           [date, elevation, geometry.coordinates, geomet...
(10.91133038, 46.06625667)           [date, elevation, geometry.coordinates, geomet...
(10.91841055, 46.351598010000004)    [date, elevation, geometry.coordinates, geomet...
(10.94136931, 45.98403497)           [date, elevation, geometry.coordinates, geomet...
(10.95755811, 46.14159

In [12]:
grouped = weatherdf.groupby("elevation").apply(list)

In [13]:
grouped.count()

35